# SQL-Augmented Generation: Putting LLMs Over Databases

You can put an LLM over a database by using it to convert natural-language questions such as "How many employees does Northwind have?" into SQL queries and executing those queries against the database. You can also automate the process using function calling. The following examples use `GPT-4o` to expose information in Microsoft's Northwind database. They require [SQLite3](https://www.sqlite.org/index.html).

Begin by using SQLite3 to connect to the database and list the names of all Northwind employees:

In [1]:
import sqlite3

connection = sqlite3.connect('Data/northwind.db')
cursor = connection.cursor()
cursor.execute('SELECT * FROM Employees')
rows = cursor.fetchall()

for row in rows:
    print(row[2] + ' ' + row[1])

Nancy Davolio
Andrew Fuller
Janet Leverling
Margaret Peacock
Steven Buchanan
Michael Suyama
Robert King
Laura Callahan
Anne Dodsworth


Now use `GPT-4o` to convert a question into a SQL query. Observe that the prompt includes a CREATE TABLE statement that details the structure of the Employees table to the LLM. Also note that the "Photos" field containing employees photos is omitted from the schema. That field is problematic if used in a query because its contents aren't readily converted to JSON:

In [ ]:
from openai import OpenAI

client = OpenAI(api_key='OPENAI_API_KEY')
question = 'How many employees does Northwind have?'

prompt = f'''
    Generate a well-formed SQLite query from the prompt below. Return
    the SQL only. Do not use markdown formatting, and do not use SELECT *.
    
    PROMPT: {question}
    
    Assume the database has a table named Employees that is defined as follows:
    
    CREATE TABLE [Employees]
    (
        [EmployeeID] INTEGER PRIMARY KEY AUTOINCREMENT,
        [LastName] TEXT,
        [FirstName] TEXT,
        [Title] TEXT,
        [TitleOfCourtesy] TEXT,
        [BirthDate] DATE,
        [HireDate] DATE,
        [Address] TEXT,
        [City] TEXT,
        [Region] TEXT,
        [PostalCode] TEXT,
        [Country] TEXT,
        [HomePhone] TEXT,
        [Extension] TEXT,
        [Notes] TEXT,
        [ReportsTo] INTEGER
    )
    '''

messages = [{ 'role': 'user', 'content': prompt }]

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages,
    temperature=0
)

sql = response.choices[0].message.content
print(sql)

SELECT COUNT(EmployeeID) AS NumberOfEmployees FROM Employees;


Execute the query against the database:

In [3]:
cursor.execute(sql)
result = cursor.fetchall()
print(result)

[(9,)]


Pass the question and the query results to the LLM and ask it to phrase an answer in natural language:

In [4]:
prompt = f'''
    Given the following question and query result, phrase the answer
    in terms that a human can understand.
    
    QUESTION: {question}
    
    RESULT: {result}
    '''

messages = [{ 'role': 'user', 'content': prompt }]

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages
)

print(response.choices[0].message.content)

Northwind has a total of 9 employees.


Now let's try a question that involves the Products table:

In [5]:
question = 'Which products have been discontinued but are currently in stock?'

prompt = f'''
    Generate a well-formed SQLite query from the prompt below. Return
    the SQL only. Do not use markdown formatting, and do not use SELECT *.
    
    PROMPT: {question}
    
    Assume the database has a table named Products that is defined as follows:
    
    CREATE TABLE [Products]
    (
        [ProductID] INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        [ProductName] TEXT NOT NULL,
        [SupplierID] INTEGER,
        [CategoryID] INTEGER,
        [QuantityPerUnit] TEXT,
        [UnitPrice] NUMERIC DEFAULT 0,
        [UnitsInStock] INTEGER DEFAULT 0,
        [UnitsOnOrder] INTEGER DEFAULT 0,
        [ReorderLevel] INTEGER DEFAULT 0,
        [Discontinued] TEXT NOT NULL DEFAULT '0'
    )
    '''

messages = [{ 'role': 'user', 'content': prompt }]

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages,
    temperature=0
)

sql = response.choices[0].message.content
print(sql)

SELECT ProductID, ProductName, SupplierID, CategoryID, QuantityPerUnit, UnitPrice, UnitsInStock, UnitsOnOrder, ReorderLevel
FROM Products
WHERE Discontinued = '1' AND UnitsInStock > 0;


Execute the query against the database:

In [6]:
cursor.execute(sql)
result = cursor.fetchall()
print(result)

[(9, 'Mishi Kobe Niku', 4, 6, '18 - 500 g pkgs.', 97, 29, 0, 0), (24, 'Guaraná Fantástica', 10, 1, '12 - 355 ml cans', 4.5, 20, 0, 0), (28, 'Rössle Sauerkraut', 12, 7, '25 - 825 g cans', 45.6, 26, 0, 0), (42, 'Singaporean Hokkien Fried Mee', 20, 5, '32 - 1 kg pkgs.', 14, 26, 0, 0)]


Ask `GPT-4o` the phrase the answer using natural language:

In [7]:
prompt = f'''
    Given the following question and query result, phrase the answer
    in terms that a human can understand.
    
    QUESTION: {question}
    
    RESULT: {result}
    '''

messages = [{ 'role': 'user', 'content': prompt }]

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages
)

print(response.choices[0].message.content)

The following products have been discontinued but are still available in stock:

1. **Mishi Kobe Niku** - There are 97 units in stock.
2. **Guaraná Fantástica** - There are 4.5 units in stock.
3. **Rössle Sauerkraut** - There are 45.6 units in stock.
4. **Singaporean Hokkien Fried Mee** - There are 14 units in stock.


Do it again, but this time ask the LLM to include the number of units that are in stock for each item that has been discontinued as well as the combined value of all the units:

In [8]:
question = '''
    Which products have been discontinued but are currently in stock,
    how many of each is in stock, and what is the combined value of all
    the products?
    '''

prompt = f'''
    Generate a well-formed SQLite query from the prompt below. Return
    the SQL only. Do not use markdown formatting, and do not use SELECT *.
    
    PROMPT: {question}
    
    Assume the database has a table named Products that is defined as follows:
    
    CREATE TABLE [Products]
    (
        [ProductID] INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        [ProductName] TEXT NOT NULL,
        [SupplierID] INTEGER,
        [CategoryID] INTEGER,
        [QuantityPerUnit] TEXT,
        [UnitPrice] NUMERIC DEFAULT 0,
        [UnitsInStock] INTEGER DEFAULT 0,
        [UnitsOnOrder] INTEGER DEFAULT 0,
        [ReorderLevel] INTEGER DEFAULT 0,
        [Discontinued] TEXT NOT NULL DEFAULT '0'
    )
    '''

messages = [
    { 'role': 'user', 'content': prompt }
]

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages,
    temperature=0
)

sql = response.choices[0].message.content
print(sql)

SELECT ProductName, UnitsInStock, (UnitsInStock * UnitPrice) AS CombinedValue
FROM Products
WHERE Discontinued = '1' AND UnitsInStock > 0;


In [9]:
cursor.execute(sql)
result = cursor.fetchall()
print(result)

[('Mishi Kobe Niku', 29, 2813), ('Guaraná Fantástica', 20, 90.0), ('Rössle Sauerkraut', 26, 1185.6000000000001), ('Singaporean Hokkien Fried Mee', 26, 364)]


In [10]:
prompt = f'''
    Given the following question and query result, phrase the answer
    in terms that a human can understand. Express monetary amounts in
    dollars.
    
    QUESTION: {question}
    
    RESULT: {result}
    '''

messages = [{ 'role': 'user', 'content': prompt }]

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages
)

print(response.choices[0].message.content)

There are four discontinued products currently in stock. 

1. "Mishi Kobe Niku" has 29 units available, with a total value of $2,813.
2. "Guaraná Fantástica" has 20 units available, with a total value of $90.
3. "Rössle Sauerkraut" has 26 units available, with a total value of $1,185.60.
4. "Singaporean Hokkien Fried Mee" has 26 units available, with a total value of $364.

The combined value of all these discontinued products in stock is $4,452.60.


Queries can target multiple tables, too. Let's define a `text2sql` function that's aware of all the tables in the Northwind database and use it from this point on to generate queries. For good measure, we'll strip markdown characters from the output in case `GPT-4o` ignores our instruction to not return markdown:

In [ ]:
import re

def text2sql(text):
    prompt = f'''
        Generate a well-formed SQLite query from the prompt below. Return
        the SQL only. Do not use markdown formatting, and do not use SELECT *.

        PROMPT: {text}
    
        The database targeted by the query contains the following tables:

        CREATE TABLE [Categories]
        (
            [CategoryID] INTEGER PRIMARY KEY AUTOINCREMENT,
            [CategoryName] TEXT,
            [Description] TEXT
        )

        CREATE TABLE [Customers]
        (
            [CustomerID] TEXT,
            [CompanyName] TEXT,
            [ContactName] TEXT,
            [ContactTitle] TEXT,
            [Address] TEXT,
            [City] TEXT,
            [Region] TEXT,
            [PostalCode] TEXT,
            [Country] TEXT,
            [Phone] TEXT,
            [Fax] TEXT,
            PRIMARY KEY (`CustomerID`)
        )

        CREATE TABLE [Employees]
        (
            [EmployeeID] INTEGER PRIMARY KEY AUTOINCREMENT,
            [LastName] TEXT,
            [FirstName] TEXT,
            [Title] TEXT,
            [TitleOfCourtesy] TEXT,
            [BirthDate] DATE,
            [HireDate] DATE,
            [Address] TEXT,
            [City] TEXT,
            [Region] TEXT,
            [PostalCode] TEXT,
            [Country] TEXT,
            [HomePhone] TEXT,
            [Extension] TEXT,
            [Notes] TEXT,
            [ReportsTo] INTEGER,
            FOREIGN KEY ([ReportsTo]) REFERENCES [Employees] ([EmployeeID]) 
        )

        CREATE TABLE [Shippers]
        (
            [ShipperID] INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            [CompanyName] TEXT NOT NULL,
            [Phone] TEXT
        )

        CREATE TABLE [Suppliers]
        (
            [SupplierID] INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            [CompanyName] TEXT NOT NULL,
            [ContactName] TEXT,
            [ContactTitle] TEXT,
            [Address] TEXT,
            [City] TEXT,
            [Region] TEXT,
            [PostalCode] TEXT,
            [Country] TEXT,
            [Phone] TEXT,
            [Fax] TEXT,
            [HomePage] TEXT
        )

        CREATE TABLE [Products]
        (
            [ProductID] INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            [ProductName] TEXT NOT NULL,
            [SupplierID] INTEGER,
            [CategoryID] INTEGER,
            [QuantityPerUnit] TEXT,
            [UnitPrice] NUMERIC DEFAULT 0,
            [UnitsInStock] INTEGER DEFAULT 0,
            [UnitsOnOrder] INTEGER DEFAULT 0,
            [ReorderLevel] INTEGER DEFAULT 0,
            [Discontinued] TEXT NOT NULL DEFAULT '0',
            FOREIGN KEY ([CategoryID]) REFERENCES [Categories] ([CategoryID]),
            FOREIGN KEY ([SupplierID]) REFERENCES [Suppliers] ([SupplierID])
        )

        CREATE TABLE [Orders]
        (
            [OrderID] INTEGER PRIMARY KEY AUTOINCREMENT,
            [CustomerID] INTEGER,
            [EmployeeID] INTEGER,
            [OrderDate] DATETIME,
            [ShipperID] INTEGER,
            FOREIGN KEY (EmployeeID) REFERENCES Employees (EmployeeID),
            FOREIGN KEY (CustomerID) REFERENCES Customers (CustomerID),
            FOREIGN KEY (ShipperID) REFERENCES Shippers (ShipperID)
        );

        CREATE TABLE [Order Details]
        (
            [OrderID] INTEGER NOT NULL,
            [ProductID] INTEGER NOT NULL,
            [UnitPrice] NUMERIC NOT NULL DEFAULT 0,
            [Quantity] INTEGER NOT NULL DEFAULT 1,
            [Discount] REAL NOT NULL DEFAULT 0,
            PRIMARY KEY ("OrderID", "ProductID"),
            FOREIGN KEY ([OrderID]) REFERENCES [Orders] ([OrderID]),
            FOREIGN KEY ([ProductID]) REFERENCES [Products] ([ProductID]) 
        )
        '''

    messages = [
        {
            'role': 'system',
            'content': 'You are a database expert who can convert questions into SQL queries'
        },
        {
            'role': 'user',
            'content': prompt
        }
    ]

    client = OpenAI(api_key='OPENAI_API_KEY')
    
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        temperature=0
    )

    sql = response.choices[0].message.content

    # Strip markdown characters if present
    pattern = r'^```[\w]*\n|\n```$'
    return re.sub(pattern, '', sql, flags=re.MULTILINE)

Let's also define a helper function for executing database queries:

In [12]:
def execute_sql(sql):
    connection = sqlite3.connect('data/northwind.db')
    cursor = connection.cursor()
    cursor.execute(sql)
    rows = cursor.fetchall()
    return rows

Now let's use these functions to ask a question that requires data from multiple tables. Start by generating the query:

In [13]:
question = 'List all products that are currently out of stock and the suppliers for those products.'
sql = text2sql(question)
print(sql)

SELECT Products.ProductName, Suppliers.CompanyName
FROM Products
JOIN Suppliers ON Products.SupplierID = Suppliers.SupplierID
WHERE Products.UnitsInStock = 0;


Execute the query and ask `GPT-4o` to phrase the results:

In [14]:
result = execute_sql(sql)

prompt = f'''
    Given the following question and query result, phrase the answer
    in terms that a human can understand.
    
    QUESTION: {question}
    
    RESULT: {result}
    '''

messages = [{ 'role': 'user', 'content': prompt }]

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages
)

print(response.choices[0].message.content)

The following products are currently out of stock, along with their respective suppliers: "Chef Anton's Gumbo Mix" from New Orleans Cajun Delights, "Alice Mutton" from Pavlova, Ltd., "Thüringer Rostbratwurst" from Plutzer Lebensmittelgroßmärkte AG, "Gorgonzola Telino" from Formaggi Fortini s.r.l., and "Perth Pasties" from G'day, Mate.


You can even use markdown formatting to enhance the output:

In [15]:
from IPython.display import Markdown, display

prompt = f'''
    Given the following question and query result, phrase the answer in
    terms that a human can understand. Format the results in markdown and
    include a markdown table if appropriate.
    
    QUESTION: {question}
    
    RESULT: {result}
    '''

messages = [{ 'role': 'user', 'content': prompt }]

response = client.chat.completions.create(
    model='gpt-4o',
    messages=messages
)

display(Markdown(response.choices[0].message.content))

Certainly! Below is a list of products that are currently out of stock along with their respective suppliers:

| Product                  | Supplier                          |
|--------------------------|-----------------------------------|
| Chef Anton's Gumbo Mix   | New Orleans Cajun Delights        |
| Alice Mutton             | Pavlova, Ltd.                     |
| Thüringer Rostbratwurst  | Plutzer Lebensmittelgroßmärkte AG |
| Gorgonzola Telino        | Formaggi Fortini s.r.l.           |
| Perth Pasties            | G'day, Mate                       |

## Combine with function calling

Rather than repeatedly generate SQL queries, execute them, and pass the query results to the LLM for phrasing, let's roll all of it up into a function and expose it to the LLM using function calling. Start by defining a function that can be called to query the Northwind database:

In [16]:
def query_database(input):
    sql = text2sql(input)
    result = execute_sql(sql)
    return json.dumps(result)

Next, describe the function in JSON:

In [17]:
database_tool = {
    'type': 'function',
    'function': {
        'name': 'query_database',
        'description': '''
            Queries the Northwind database to answer a question or respond
            to a command. Northwind contains information about sales, products,
            orders, and employees of a fictitious company named Northwind Traders.
            The database contains the following tables:

            Categories - Information about product categories
            Customers - Information about customers who purchase Northwind products
            Employees - Information about employees of Northwind Traders
            Shippers - Information about companies that ship Northwind products
            Suppliers - Information about suppliers of Northwind products
            Products - Information about the products that Northwind sells
            Orders - Information about orders placed by Northwind customers
            OrderDetails - Information about order details such as products and quantities
            ''',
        'parameters': {
            'type': 'object',
            'properties': {
                'input': {
                    'type': 'string',
                    'description': 'Input from the user'
                },
            },
            'required': ['input']
        }
    }
}

Now define a `chat` function that's aware of the `query_database` function and instruct the LLM to answer "I don't know" if asked a question whose answer can't be found in Northwind:

In [18]:
import json

def chat(input, messages=None):
    if not messages:
        instructions = '''
            You are a helpful assistant who can answer questions from the Northwind database.
            Northwind contains information about sales, products, orders, and employees of a
            fictitious company named Northwind Traders. The database contains the following
            tables:

            Categories - Information about product categories
            Customers - Information about customers who purchase Northwind products
            Employees - Information about employees of Northwind Traders
            Shippers - Information about companies that ship Northwind products
            Suppliers - Information about suppliers of Northwind products
            Products - Information about the products that Northwind sells
            Orders - Information about orders placed by Northwind customers
            OrderDetails - Information about order details such as products and quantities  

            Only answer questions that can be answered by querying the database. If asked a
            question that can't be answered by a database query, say "I don't know."
            '''

        messages = [{ 'role': 'system', 'content': instructions }]
    
    message = { 'role': 'user', 'content': input }
    messages.append(message)

    # Call the LLM  and make it aware of the database tool
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        tools=[database_tool]
    )
    
    # If one or more tool calls are required, execute them
    if response.choices[0].message.tool_calls:
        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
    
            if function_name == 'query_database':
                input = json.loads(tool_call.function.arguments)['input']
                output = query_database(input)

                # Append the function output to the messages list
                messages.append({ 'role': 'function', 'name': function_name, 'content': output })
            else:
                raise Exception('Invalid function name')
    
        # Pass the function output to the LLM
        response = client.chat.completions.create(
            model='gpt-4o',
            messages=messages
        )
    
    # Return a message thread containing the LLM's response
    messages.append({ 'role': 'assistant', 'content': response.choices[0].message.content })
    return messages

Use the `chat` function to get an answer to a question:

In [19]:
messages = chat('List all products that are currently out of stock and the suppliers for those products.')
print(messages[-1]['content'])

Here are the products that are currently out of stock along with their suppliers:

1. Chef Anton's Gumbo Mix - New Orleans Cajun Delights
2. Alice Mutton - Pavlova, Ltd.
3. Thüringer Rostbratwurst - Plutzer Lebensmittelgroßmärkte AG
4. Gorgonzola Telino - Formaggi Fortini s.r.l.
5. Perth Pasties - G'day, Mate


Try another question:

In [20]:
messages = chat('List all products that are currently out of stock and how many of each are currently on order.')
print(messages[-1]['content'])

Here are the products that are currently out of stock and their respective quantities on order:

- Alice Mutton: 978 units on order
- Chef Anton's Gumbo Mix: 298 units on order
- Gorgonzola Telino: 1397 units on order
- Perth Pasties: 722 units on order
- Thüringer Rostbratwurst: 746 units on order


And then another:

In [21]:
messages = chat("What are Northwind's most popular products, and how many of each have been sold?")
print(messages[-1]['content'])

Northwind's most popular products and the quantities sold are as follows:

1. Camembert Pierrot - 1,577 units
2. Raclette Courdavault - 1,496 units
3. Gorgonzola Telino - 1,397 units
4. Gnocchi di nonna Alice - 1,263 units
5. Pavlova - 1,158 units
6. Rhönbräu Klosterbier - 1,155 units
7. Guaraná Fantástica - 1,125 units
8. Boston Crab Meat - 1,103 units
9. Tarte au sucre - 1,083 units
10. Flotemysost - 1,057 units


Confirm that the LLM won't answer generic questions that have nothing to do with Northwind:

In [22]:
messages = chat('Why is the sky blue?')
print(messages[-1]['content'])

I don't know.


If you would like for the LLM to answer questions unrelated to the Northwind database, you can remove the text from the system prompt that instructs the LLM to say "I don't know" if a question can't be answered with a database query.